In [1]:
import pandas as pd
import numpy as np
from glob import glob
from tqdm.notebook import tqdm

BASE = "/media/giani/Gianicosas/Magister/Proyecto/raw_data"

# Proceso de matricula

## Datos

Se comparará la cantidad de alumnos por carrera por año para validación de datos

In [2]:
# Fuente: https://www.mifuturo.cl/bases-de-datos-de-matriculados/
proceso = pd.read_csv(f"{BASE}/OFICIAL_WEB_PROCESO_MAT_2007_AL_2022_24_06_2022_E.csv", encoding="latin-1", sep=";")
proceso.head()

/tmp/ipykernel_12671/618280487.py:2: DtypeWarning: Columns (27) have mixed types. Specify dtype option on import or set low_memory=False.
  proceso = pd.read_csv(f"{BASE}/OFICIAL_WEB_PROCESO_MAT_2007_AL_2022_24_06_2022_E.csv", encoding="latin-1", sep=";")


,AÑO,TOTAL MATRICULADOS,MATRICULADOS MUJERES POR PROGRAMA,MATRICULADOS HOMBRES POR PROGRAMA,TOTAL MATRICULADOS PRIMER AÑO,MATRICULADOS MUJERES PRIMER AÑO,MATRICULADOS HOMBRES PRIMER AÑO,CLASIFICACIÓN INSTITUCIÓN NIVEL 1,CLASIFICACIÓN INSTITUCIÓN NIVEL 2,CLASIFICACIÓN INSTITUCIÓN NIVEL 3,...,TES PARTICULAR SUBVENCIONADO,TES PARTICULAR PAGADO,TES CORP. DE ADMINISTRACIÓN DELEGADA,TES SERVICIO LOCAL EDUCACION,TOTAL TES,% COBERTURA TES,TIPO ESTABLECIMIENTO HC,TIPO ESTABLECIMIENTO TP,CLAS_EST ADULTO,CLAS_EST JOVEN
0,MAT_2022,2,2.0,NaN,NaN,NaN,NaN,Institutos Profesionales,Institutos Profesionales,Institutos Profesionales,...,2.0,NaN,NaN,NaN,2,"100,0%",2.0,NaN,NaN,2.0
1,MAT_2022,10,8.0,2.0,NaN,NaN,NaN,Institutos Profesionales,Institutos Profesionales,Institutos Profesionales,...,3.0,2.0,NaN,NaN,9,"90,0%",4.0,5.0,2.0,7.0
2,MAT_2022,34,NaN,34.0,NaN,NaN,NaN,Institutos Profesionales,Institutos Profesionales,Institutos Profesionales,...,14.0,3.0,NaN,NaN,26,"76,5%",17.0,9.0,4.0,22.0
3,MAT_2022,10,NaN,10.0,NaN,NaN,NaN,Institutos Profesionales,Institutos Profesionales,Institutos Profesionales,...,4.0,NaN,NaN,NaN,10,"100,0%",6.0,4.0,1.0,9.0
4,MAT_2022,76,4.0,72.0,NaN,NaN,NaN,Institutos Profesionales,Institutos Profesionales,Institutos Profesionales,...,29.0,6.0,4.0,NaN,71,"93,4%",30.0,41.0,4.0,67.0


In [3]:
proceso = proceso[proceso["CLASIFICACIÓN INSTITUCIÓN NIVEL 1"].isin(['Institutos Profesionales', 'Centros de Formación Técnica'])]

In [4]:
proceso["AÑO"].value_counts().sort_index(ascending=False)

MAT_2022    7302
MAT_2021    7168
MAT_2020    6576
MAT_2019    6552
MAT_2018    6629
MAT_2017    6772
MAT_2016    6934
MAT_2015    7005
MAT_2014    7006
MAT_2013    6730
MAT_2012    6047
MAT_2011    5488
MAT_2010    5143
MAT_2009    4786
MAT_2008    4124
MAT_2007    3645
Name: AÑO, dtype: int64

# Matriculados

### Todas las columnas
```python
# pd.read_csv(f"{BASE}/20220719_Matrícula_Ed_Superior_2022_PUBL_MRUN.csv", nrows=5, sep=";").columns
[
    'cat_periodo', 'id', 'codigo_unico', 'mrun', 'gen_alu', 'fec_nac_alu', 'rango_edad', 'anio_ing_carr_ori',
    'sem_ing_carr_ori', 'anio_ing_carr_act', 'sem_ing_carr_act', 'anio_ing_carr_act', 'tipo_inst_1', 'tipo_inst_2', 'tipo_inst_3',
    'cod_inst', 'nomb_inst', 'cod_sede', 'nomb_sede', 'cod_carrera', 'nomb_carrera', 'modalidad', 'jornada',
    'version', 'tipo_plan_carr', 'dur_estudio_carr', 'dur_proceso_tit', 'dur_total_carr', 'region_sede',
    'provincia_sede', 'comuna_sede', 'nivel_global', 'nivel_carrera_1', 'nivel_carrera_2', 'requisito_ingreso',
    'vigencia_carrera', 'formato_valores', 'valor_matricula', 'valor_arancel', 'codigo_demre', 'area_conocimiento',
    'cine_f_97_area', 'cine_f_97_subarea', 'area_carrera_generica', 'cine_f_13_area', 'cine_f_13_subarea',
    'acreditada_carr', 'acreditada_inst', 'acre_inst_desde_hasta', 'acre_inst_anio', 'costo_proceso_titulacion',
    'costo_obtencion_titulo_diploma', 'forma_ingreso'
]
```

### Cols para determinar vector objetivo
```python
['cat_periodo', 'codigo_unico', 'mrun', 'anio_ing_carr_ori', 'anio_ing_carr_act', 'tipo_inst_1', 'dur_estudio_carr']
```

### Tipos de instituciones
```python
# Salida: ['Universidades', 'Institutos Profesionales', 'Centros de Formación Técnica']
pd.concat([
    pd.read_csv(f, sep=";", usecols=["tipo_inst_1"])
    for f in tqdm(glob(f"{BASE}/20220719_Matrícula_Ed_Superior_*.csv"), total=16)
])["tipo_inst_1"].unique()
instituciones = ['Institutos Profesionales', 'Centros de Formación Técnica']
```

## Lectura de datos matriculados

Fuente: https://datosabiertos.mineduc.cl/matricula-en-educacion-superior/

In [5]:
def iter_df_read(file, col_name, condition_list, chunksize=1000, **kwargs):
    iter_csv = pd.read_csv(file, iterator=True, chunksize=chunksize, **kwargs)
    return pd.concat([chunk[chunk[col_name].isin(condition_list)] for chunk in iter_csv])

In [6]:
cols_time = ['cat_periodo', 'codigo_unico', 'mrun', 'anio_ing_carr_ori', 'anio_ing_carr_act', 'tipo_inst_1', 'dur_estudio_carr']
instituciones = ['Institutos Profesionales', 'Centros de Formación Técnica']

matriculados = pd.concat([
    iter_df_read(f, "tipo_inst_1", instituciones, sep=";", usecols=cols_time)
    for f in tqdm(glob(f"{BASE}/post_2015/20220719_Matrícula_Ed_Superior_*.csv"), total=8)
])
matriculados.head()

  0%|          | 0/8 [00:00<?, ?it/s]

,cat_periodo,codigo_unico,mrun,anio_ing_carr_ori,anio_ing_carr_act,tipo_inst_1,dur_estudio_carr
0,2015,I374S1C9J1V1,5.0,2015,NaN,Centros de Formación Técnica,5
1,2015,I498S6C132J2V1,37.0,2015,NaN,Centros de Formación Técnica,5
5,2015,I111S12C104J1V1,107.0,2013,NaN,Institutos Profesionales,9
7,2015,I111S14C309J2V1,118.0,2015,NaN,Institutos Profesionales,5
9,2015,I260S44C120J2V1,184.0,2014,NaN,Centros de Formación Técnica,5


### Información del DataFrame

In [8]:
matriculados.shape

(4133070, 7)

In [9]:
matriculados.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4133070 entries, 0 to 1301920
Data columns (total 7 columns):
 #   Column             Dtype  
---  ------             -----  
 0   cat_periodo        int64  
 1   codigo_unico       object 
 2   mrun               float64
 3   anio_ing_carr_ori  int64  
 4   anio_ing_carr_act  float64
 5   tipo_inst_1        object 
 6   dur_estudio_carr   int64  
dtypes: float64(2), int64(3), object(2)
memory usage: 252.3+ MB


### Exploración de nulos en año de ingreso a carrera actual

In [10]:
matriculados.isna().sum()

cat_periodo                0
codigo_unico               0
mrun                    1461
anio_ing_carr_ori          0
anio_ing_carr_act    1193783
tipo_inst_1                0
dur_estudio_carr           0
dtype: int64

In [11]:
for a in range(2015, 2023):
    t = matriculados[matriculados['cat_periodo']==a]['anio_ing_carr_act']
    print(f"Cantidad de datos Año {a}: {t.shape[0]}. Cantidad nulos: {t.isna().sum()}. Porcentaje nulos: {round(t.isna().sum() / t.shape[0] * 100, 2)}%")

Cantidad de datos Año 2015: 525348. Cantidad nulos: 348085. Porcentaje nulos: 66.26%
Cantidad de datos Año 2016: 526387. Cantidad nulos: 432112. Porcentaje nulos: 82.09%
Cantidad de datos Año 2017: 514142. Cantidad nulos: 413586. Porcentaje nulos: 80.44%
Cantidad de datos Año 2018: 511638. Cantidad nulos: 0. Porcentaje nulos: 0.0%
Cantidad de datos Año 2019: 519361. Cantidad nulos: 0. Porcentaje nulos: 0.0%
Cantidad de datos Año 2020: 492383. Cantidad nulos: 0. Porcentaje nulos: 0.0%
Cantidad de datos Año 2021: 514348. Cantidad nulos: 0. Porcentaje nulos: 0.0%
Cantidad de datos Año 2022: 529463. Cantidad nulos: 0. Porcentaje nulos: 0.0%


### Validación: Código único

- Se corrobora que un código único puede estar asociado a distintos nombres de carrera, pero se asumirá que todos los nombres diferentes hacen referencia a la misma.
- Todos los códigos únicos repetidos pertenecen a la misma institución
- Se observa un ejemplo

In [12]:
proceso.groupby(["CÓDIGO CARRERA"]).agg({"NOMBRE INSTITUCIÓN": pd.Series.nunique, "NOMBRE CARRERA": pd.Series.nunique}).reset_index().sort_values(by=["NOMBRE CARRERA"], ascending=False).head()

,CÓDIGO CARRERA,NOMBRE INSTITUCIÓN,NOMBRE CARRERA
4011,I113S8C61J1V1,1,4
3817,I113S5C61J2V1,1,4
3963,I113S8C162J2V1,1,4
15817,I331S1C100J1V1,1,4
15818,I331S1C100J2V1,1,4
